In [1]:
import pandas as pd 
import psycopg2
import yfinance as yf 

In [11]:
goo = yf.Ticker("AAPL")
hist = goo.history(period="1y") #Elegir desde el año pasado para atrás
hist["Date"]=hist.index
hist=hist.reset_index(drop=True)
hist.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Date'],
      dtype='object')

In [24]:
hist=hist.rename(columns={"Open":"O", "Close":"C", "Stock Splits": "Stock_Splits" })

In [25]:
hist #Data a traspasar a redshift 

,O,High,Low,C,Volume,Dividends,Stock_Splits,Date
0,137.394939,142.226278,136.878001,141.609924,114311700,0.0,0.0,2022-10-03 00:00:00-04:00
1,144.174691,145.357676,143.409228,145.238388,87830100,0.0,0.0,2022-10-04 00:00:00-04:00
2,143.220375,146.510853,142.166614,145.536621,79471000,0.0,0.0,2022-10-05 00:00:00-04:00
3,144.950090,146.669884,144.363574,144.572327,68402200,0.0,0.0,2022-10-06 00:00:00-04:00
4,141.699388,142.256098,138.627615,139.263840,85925600,0.0,0.0,2022-10-07 00:00:00-04:00
...,...,...,...,...,...,...,...,...
246,174.820007,175.199997,171.660004,171.960007,64588900,0.0,0.0,2023-09-26 00:00:00-04:00
247,172.619995,173.039993,169.050003,170.429993,66921800,0.0,0.0,2023-09-27 00:00:00-04:00
248,169.339996,172.029999,167.619995,170.690002,56294400,0.0,0.0,2023-09-28 00:00:00-04:00
249,172.020004,173.070007,170.339996,171.210007,51814200,0.0,0.0,2023-09-29 00:00:00-04:00


In [26]:
hist.columns

Index(['O', 'High', 'Low', 'C', 'Volume', 'Dividends', 'Stock_Splits', 'Date'], dtype='object')

CREATE TABLE AAPL_PRICES (
    O DECIMAL(10, 2),
    High DECIMAL(10, 2),
    Low DECIMAL(10, 2),
    C DECIMAL(10, 2),
    Volume BIGINT,
    Dividends DECIMAL(10, 2),
    Stock_Splits DECIMAL(10, 2),
    Date TIMESTAMP
);

In [28]:
#Conexión a RedShift 
try:
    conn= psycopg2.connect(
        host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com",
        dbname= "data-engineer-database",
        user="frnmn20_coderhouse",
        password= "3076WtdbVk",
        port= "5439"
    )
    print ("Connected to Redshift successfully!")
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


In [29]:
cur = conn.cursor()
cur.execute("select *from aapl_prices")
results= cur.fetchall
results

<function cursor.fetchall>

In [30]:
from psycopg2.extras import execute_values
cur = conn.cursor()
table_name= "aapl_prices"
columns = ['O', 'High', 'Low', 'C', 'Volume', 'Dividends', 'Stock_Splits', 'Date']
values = [tuple(x) for x in hist.to_numpy()]
insert_sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES %s"
cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute("COMMIT")

select *from aapl_prices 